# Comparison between meshgrid and einmesh

In this example we are going to show some of the justifying examples where i think einmesh makes code more readable, more maintainable and just simply shorter than the current standard meshgrid function. 



In [ ]:
from einmesh.numpy import einmesh

A classic exam

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

x = np.arange(-5, 5, 1)
y = np.arange(-5, 5, 1)
xx, yy = np.meshgrid(x, y, sparse=True)

z = np.sin(xx**2 + yy**2) / (xx**2 + yy**2)
h = plt.contourf(x, y, z)  # often we plot something then

Immediatly we see the weirdness of the current meshgrid function. Firstly, we define two variables to make our plot over, x and y, we then immediatly feed this to the meshgrid function and define two new variables, which turns out are the actual x and y we want for our computation, and call them xx and yy (as if to say that these are more x and y than x and y). 

We 



We first create x and y variables, which are never used again. But because they have

In [ ]:
from einmesh import RangeSpace

x, y = einmesh("x y", x=RangeSpace(-5, 5), y=RangeSpace(-5, 5))
z = np.sin(x**2 + y**2) / (x**2 + y**2)
h = plt.contourf(x, y, z)  # often we plot something then

Just searching for meshgrid on Github gives a representation of what kinds of things people use the function for

https://github.com/frozoul/4K-NeRF/blob/420897a6f5a473fba258dc428981759f91e96cdb/run.py#L263

In [ ]:
@torch.no_grad()
def compute_bbox_by_coarse_geo(model_class, model_path, thres):
    print("compute_bbox_by_coarse_geo: start")
    eps_time = time.time()
    model = utils.load_model(model_class, model_path)
    interp = torch.stack(
        torch.meshgrid(
            torch.linspace(0, 1, model.world_size[0]),
            torch.linspace(0, 1, model.world_size[1]),
            torch.linspace(0, 1, model.world_size[2]),
        ),
        -1,
    )
    dense_xyz = model.xyz_min * (1 - interp) + model.xyz_max * interp
    density = model.density(dense_xyz)
    alpha = model.activate_density(density)
    mask = alpha > thres
    active_xyz = dense_xyz[mask]
    xyz_min = active_xyz.amin(0)
    xyz_max = active_xyz.amax(0)
    print("compute_bbox_by_coarse_geo: xyz_min", xyz_min)
    print("compute_bbox_by_coarse_geo: xyz_max", xyz_max)
    eps_time = time.time() - eps_time
    print("compute_bbox_by_coarse_geo: finish (eps time:", eps_time, "secs)")
    return xyz_min, xyz_max



https://github.com/eaglewarrior/Machine-learning/blob/e419e77a0db226901a58c98eed0fe988efd009b3/pca.py#L45

In [ ]:
# Visualising the Training set results
from matplotlib.colors import ListedColormap

X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(
    np.arange(start=X_set[:, 0].min() - 1, stop=X_set[:, 0].max() + 1, step=0.01),
    np.arange(start=X_set[:, 1].min() - 1, stop=X_set[:, 1].max() + 1, step=0.01),
)
plt.contourf(
    X1,
    X2,
    classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
    alpha=0.75,
    cmap=ListedColormap(("red", "green", "blue")),
)

In [ ]:
import torch
from torch import nn

transformer_model = nn.Transformer(nhead=2, num_encoder_layers=2, d_model=16)
src = torch.rand((10, 16, 16))
tgt = torch.rand((20, 16, 16))
out = transformer_model(src, tgt)

print(out.shape)

In [ ]:
ni, nj, nk = 4, 5, 6
si, sj, sk = 20, 10, 1

xcorn = einmesh("* x", x=RangeSpace(0, (ni + 1) * si, si))
print(xcorn.shape)

xcorn = np.arange(0, (ni + 1) * si, si)
print(xcorn.shape)
xcorn = np.repeat(xcorn, 2)
xcorn = xcorn[1:-1]
xcorn = np.tile(xcorn, 4 * nj * nk)
print(xcorn.shape)

In [ ]:
import numpy as np
import pyvista as pv

ni, nj, nk = 4, 5, 6
si, sj, sk = 20, 10, 1

xcorn = einmesh("x", x=RangeSpace(0, (ni + 1) * si, si))


xcorn = np.arange(0, (ni + 1) * si, si)
xcorn = np.repeat(xcorn, 2)
xcorn = xcorn[1:-1]
xcorn = np.tile(xcorn, 4 * nj * nk)

ycorn = np.arange(0, (nj + 1) * sj, sj)
ycorn = np.repeat(ycorn, 2)
ycorn = ycorn[1:-1]
ycorn = np.tile(ycorn, (2 * ni, 2 * nk))
ycorn = np.transpose(ycorn)
ycorn = ycorn.flatten()

zcorn = np.arange(0, (nk + 1) * sk, sk)
zcorn = np.repeat(zcorn, 2)
zcorn = zcorn[1:-1]
zcorn = np.repeat(zcorn, (4 * ni * nj))

corners = np.stack((xcorn, ycorn, zcorn))
corners = corners.transpose()

dims = np.asarray((ni, nj, nk)) + 1
grid = pv.ExplicitStructuredGrid(dims, corners)
grid = grid.compute_connectivity()
grid.plot(show_edges=True)

# Generating fake input for Deep Learning